In [1]:
import sys
sys.path.append('..')

In [2]:
import tqdm
import torch
import pickle
import warnings
import numpy as np
import pandas as pd
import torch.optim as optim
from copy import deepcopy
import plotly.express as px
import plotly.graph_objects as go

from src.data import *
from src.utils import *
from src.model import *
from src.recourse import *

warnings.filterwarnings('ignore')

In [3]:
def append_result(d, rob, loss, cost, m1_validity, wc_validity, m1_expectation, wc_expectation):
    d['Cost'].append(cost)
    d['M1 Validity'].append(m1_validity)
    d['WC Validity'].append(wc_validity)
    d['M1 Expectation'].append(m1_expectation)
    d['WC Expectation'].append(wc_expectation)
    d['J'].append(rob) 
    d['Loss'].append(loss)
    
def get_result(d, alg, seed, alpha, lamb, theta_0, theta_r):
    result = {
        'alg': alg, 
        'seed': seed,
        'alpha': alpha,
        'lambda': lamb,
        'theta_0': theta_0,
        'theta_r': theta_r
        }
    
    for key in d.keys():
        result[key] = np.mean(d[key])
    return result

In [4]:
def get_theta_adv_pga(X_0, X_r, theta_0, alpha, lamb, pga_max_iter: int = 100):
    X_0 = torch.tensor(np.stack(X_0)).float()
    X_r = torch.tensor(np.stack(X_r)).float()
    
    loss_fn = torch.nn.BCELoss(reduction='mean')
    clf_adv = LogReg(X_0.shape[1])
    optimizer = optim.Adam(clf_adv.model.parameters(), maximize=True)
    weights_min, weights_max = [], []
    bias_min, bias_max = [], []
    for module in clf_adv.model:
        if hasattr(module, 'weight'):
            module.weight.data = torch.from_numpy(deepcopy(theta_0[:-1].reshape(1,-1))).float()
            weights_min.append(deepcopy(module.weight.data - alpha))
            weights_max.append(deepcopy(module.weight.data + alpha))
            module.weight.data += torch.randn(module.weight.data.shape)
        else:
            weights_min.append(0)
            weights_max.append(0)
        if hasattr(module, 'bias'):
            module.bias.data = torch.from_numpy(theta_0[[-1]]).float()
            bias_min.append(deepcopy(module.bias.data - alpha))
            bias_max.append(deepcopy(module.bias.data + alpha))
            module.weight.data += torch.randn(module.bias.data.shape)
        else:
            bias_min.append(0)
            bias_max.append(0)
        
    for epoch in range(pga_max_iter):
        optimizer.zero_grad()
        
        f_x = clf_adv.forward(X_r)
        y_target = torch.ones(f_x.shape).float()
        bce_loss = loss_fn(f_x, y_target)
        cost = torch.dist(X_r, X_0, 1)
        loss = bce_loss + lamb*cost
        
        loss.backward()
        optimizer.step()
        
        # clamp model parameters to -alpha, alpha range
        for i, module in enumerate(clf_adv.model):
            if hasattr(module, 'weight'):
                module.weight.data = module.weight.data.clamp(weights_min[i], weights_max[i])
            if hasattr(module, 'bias'):
                module.bias.data = module.bias.data.clamp(bias_min[i], bias_max[i])
                
    
    for i, module in enumerate(clf_adv.model):
        if hasattr(module, 'weight'):
            weights_r = module.weight.data.numpy()[0]
        if hasattr(module, 'bias'):
            bias_r = module.bias.data.numpy()
            
    theta_adv = np.hstack((weights_r, bias_r))
    
    print(f'Final Loss: {loss}')
    print(f'Num Iterations: {i}')
    print(f'theta_alpha: {np.linalg.norm(theta_adv-theta_0, ord=np.inf)}')
    print()
    
    return theta_adv

In [5]:
def get_theta_adv_search(X_0, X_r, theta_0, alpha, lamb):
    
    theta_adv = deepcopy(theta_0)
    X_0 = torch.tensor(np.stack(X_0)).float()
    X_r = torch.tensor(np.stack(X_r)).float()
    
    for i in range(X_0.shape[1]):
        theta_r_min = deepcopy(theta_0)
        theta_r_max = deepcopy(theta_0)
        
        theta_r_min[i] -= alpha
        theta_r_max[i] += alpha
        weights_r_min, bias_r_min = theta_r_min[:-1], theta_r_min[[-1]]
        weights_r_max, bias_r_max = theta_r_max[:-1], theta_r_max[[-1]]
        J_min, J_max = [], []
        for xi in range(len(X_r)):
            x_0 = X_0[xi]
            x_r = X_r[xi]
            J = RecourseCost(x_0, lamb)
            j_min = J.eval(x_r, weights_r_min, bias_r_min)
            j_max = J.eval(x_r, weights_r_max, bias_r_max)
            J_min.append(j_min)
            J_max.append(j_max)
        
        
        if np.mean(J_min) > np.mean(J_max):
            theta_adv[i] -= alpha
        else:
            theta_adv[i] += alpha
    
    return theta_adv

In [6]:
def evaluate_performance(X_0, X_r, theta_0, alpha, lamb, seed, alg, method='search'):
    results = {'Cost': [], 'M1 Validity': [], 'WC Validity': [], 'M1 Expectation': [], 'WC Expectation': [], 'J': [], 'Loss': []}
    
    weights_0, bias_0 = theta_0[:-1], theta_0[[-1]]
    if method=='search':
        theta_r = get_theta_adv_search(X_0, X_r, theta_0, alpha, lamb)
    else:
        theta_r = get_theta_adv_pga(X_0, X_r, theta_0, alpha, lamb, pga_max_iter=500)
    for ti in range(len(theta_0)):
        if np.abs(theta_r[ti] - (theta_0[ti]-alpha)) < np.abs(theta_r[ti] - (theta_0[ti]+alpha)):
            theta_r[ti] = theta_0[ti]-alpha
        else:
            theta_r[ti] = theta_0[ti]+alpha
    weights_r, bias_r = theta_r[:-1], theta_r[[-1]]
        
    n = len(X_r)
    Y_0 = np.hstack((np.ones((n//2,)), np.zeros((n - n//2,))))
    
    cfr = LR()
    cfr.train(X_0, Y_0)
    cfr.model.coef_ = weights_0.reshape(1,-1)
    cfr.model.intercept_ = bias_0
    
    cfr_adv = deepcopy(cfr)
    cfr_adv.model.coef_ = weights_r.reshape(1,-1)
    cfr_adv.model.intercept_ = bias_r

    for i in tqdm.trange(n, desc=f'Eval alpha={alpha}; lambda={lamb}', colour='#0091ff'):
        x_0 = X_0[i]
        x_r = X_r[i]
        J = RecourseCost(x_0, lamb)
        
        bce_loss_opt, cost_opt, rob_opt = J.eval(x_r, weights_r, bias_r, True)
        m1_validity_opt = cfr.predict(x_r.reshape(1,-1))[0]
        m1_expectation_opt = cfr.predict_proba(x_r.reshape(1,-1))[0,1]
        
        wc_validity_opt = cfr_adv.predict(x_r.reshape(1,-1))[0]
        wc_expectation_opt = cfr_adv.predict_proba(x_r.reshape(1,-1))[0,1]
        
        append_result(results, rob_opt, bce_loss_opt, cost_opt, m1_validity_opt, wc_validity_opt, m1_expectation_opt, wc_expectation_opt)
        
    return get_result(results, alg, seed, alpha, lamb, theta_0, theta_r)

In [7]:
method_map = {
    'alg1_lamb0.05': 'Alg1 (λ=0.05)', 
    'alg1_lamb0.1': 'Alg1 (λ=0.1)',
    'alg1_lamb0.2': 'Alg1 (λ=0.2)', 
    'alg1_lamb0.3': 'Alg1 (λ=0.3)',
    'roar': 'ROAR ', 
    'rbr': 'RBR', 
    'wachter': 'WACHTER'
    }

data_map = {'synthetic': 'Synthetic', 'sba': 'Small Business Administration', 'german': 'German', 'income': 'ACS Income'}
model_map = {'lr': 'Logistic Regression', 'nn': 'Neural Network'}

In [8]:
torch.manual_seed(0)
params = {}
# 'synthetic', 'german', 'sba'
params['data'] = 'sba'
# 'lr', 'nn'
params['base_model'] = 'lr'
params['algs'] = ['alg1', 'roar']
lambdas = [0.1, 0.2, 0.3]

methods = []
for method in params['algs']:
    for li, lamb in enumerate(lambdas):
        if method == 'alg1':
            methods.append(f'{method}_lamb{lamb}')
        else:
            if li > 0:
                break
            methods.append(f'{method}')
                
results = {
    'Algorithm': [],
    'Cost': [],
    'Current Validity': [],
    'Worst Case Validity': [],
    'alpha': []
}
                
                
for li, method in enumerate(methods):
    with open(f"../results/cost_validity/{params['base_model']}_{params['data']}_{method}.pickle", 'rb') as f:
        data = pickle.load(f)

    lamb = lambdas[li%len(lambdas)]
    
    for i in range(len(data["params"])):
        alpha = data['params'][i]['delta_max']
        
        X_0 = data['x_0'][i]
        X_r = data['x_r'][i]
        
        try:
            weights_0, bias_0 = data['theta_0'][i][0]
        except:
            try:
                weights_0, bias_0 = data['theta_0']['out.weight'].data.detach(), data['theta_0']['out.bias'].data.detach()
            except:
                weights_0, bias_0 = data['theta_0'][i]        
        
        weights_0, bias_0 = weights_0[0].numpy(), bias_0.numpy()
        theta_0 = np.hstack((weights_0, bias_0))
        
        res = evaluate_performance(X_0, X_r, theta_0, alpha, lamb, i, method, 'search')
        results['Algorithm'].append(method)
        results['Cost'].append(res['Cost'])
        results['Current Validity'].append(res['M1 Validity'])
        results['Worst Case Validity'].append(res['WC Validity'])
        results['alpha'].append(alpha)
df_results = pd.DataFrame(results)

Eval alpha=0.2; lambda=0.1: 100%|██████████| 100/100 [00:00<00:00, 6254.27it/s]


In [9]:
print(f'{data_map[params["data"]]}  |  {params["base_model"].upper()}')
df_results_avg = df_results.groupby('Algorithm').mean(True)
df_results_avg[['Cost', 'Current Validity', 'Worst Case Validity']] = df_results_avg[['Cost', 'Current Validity', 'Worst Case Validity']].round(2).astype(str) + '±' + df_results.groupby('Algorithm').std(numeric_only=True)[['Cost', 'Current Validity', 'Worst Case Validity']].round(2).astype(str)

df_results_avg

Small Business Administration  |  LR


,Cost,Current Validity,Worst Case Validity,alpha
Algorithm,,,,
alg1_lamb0.1,4.54±1.37,0.98±0.03,0.94±0.01,0.1
alg1_lamb0.2,2.46±0.96,0.88±0.23,0.62±0.15,0.1
alg1_lamb0.3,1.36±0.83,0.38±0.37,0.11±0.06,0.1
roar,0.75±0.33,0.05±0.06,0.08±0.03,0.1


In [10]:
# colors = ['#1f77b4', '#17becf', '#9467bd', '#e377c2', '#2ca02c'] # Synthesis
# colors = ['#17becf', '#e377c2', '#2ca02c'] # German
colors = ['#17becf', '#9467bd', '#e377c2', '#2ca02c'] # SBA

nc = len(colors)
font_family = 'Times New Roman'
font_color = 'black'
width, height = 720, 540

symbols = ['x' for _ in range(len(lambdas))] + ['circle']
size = [7 for _ in range(len(lambdas))] + [5]

show_errors = False

fig = go.Figure()
for i, alg in enumerate(df_results['Algorithm'].unique()):
    df_alg = df_results[(df_results['Algorithm'] == alg)].sort_values(['Cost'], ascending=True).copy().reset_index(drop=True)
    x,y, mask = find_pareto(df_alg['Cost'], df_alg['Worst Case Validity'], return_index=True)
    df_alg = pd.DataFrame({'Algorithm': [method_map[alg] for _ in range(len(x))], 'Cost': x, 'Worst Case Validity': y, 'alpha': df_alg['alpha'][mask]})

    fig.add_trace(go.Scatter(
        x = df_alg['Cost'],
        y = df_alg['Worst Case Validity'],
        marker = dict(color=colors[i], symbol=symbols[i], size=size[i]),
        mode = 'lines+markers' if alg != 'wachter' else 'markers',
        name = method_map[alg],
        showlegend=True,
        customdata=df_alg['alpha'],
        hovertemplate='Cost: %{x}<br>Validity: %{y}<br>alpha: %{customdata}'
    ))

fig.update_xaxes(
    title=dict(
        text='Cost',
        font=dict(
            family=font_family,
            color=font_color,
            size=25
        )
        ), 
    showline=True, 
    mirror=True,
    linecolor='black', 
    gridcolor='lightgrey', 
    zerolinewidth=1,
    zerolinecolor='lightgrey',
    )


fig.update_yaxes(
    title=dict(
        text='Worst Case Validity',
        font=dict(
            family=font_family,
            color=font_color,
            size=25
        ), 
        ), 
    showline=True, 
    mirror=True,
    linecolor='black', 
    gridcolor='lightgrey',
    zerolinewidth=1,
    zerolinecolor='lightgrey',
    )


fig.update_layout(
    legend=dict(
        x=0.975, 
        y=0.025, 
        orientation='v',
        xanchor='right',
        font=dict(
            family=font_family,
            color=font_color,
            size=15
            ), 
        bgcolor='rgba(255, 255, 255, 0.7)',
        bordercolor='lightgrey',
        borderwidth=1,
        entrywidth=100.5,
        ),
    width=width,
    height=height,
    plot_bgcolor='white',
    paper_bgcolor='white',
    xaxis=dict(
        tickfont=dict(
            family=font_family,
            color=font_color,
            size=20,
        ),
    ),
    yaxis=dict(
        tickfont=dict(
            family=font_family,
            color=font_color,
            size=20
        ),
        range=[-0.1,1.1],
    )
)

print(f'{data_map[params["data"]]}  |  {params["base_model"].upper()}')
fig.show()

Small Business Administration  |  LR


In [11]:
# fig.write_image(f'../figs/cost_validity_tradeoff_{params["base_model"]}_{params["data"]}.pdf')